In [ ]:
from airflow import DAG

from airflow.operators.python import PythonOperator


from time import strftime,localtime
from web_scraper import *

In [ ]:
with DAG('player_box_scores_etl',default_args={'retries': 4},description='ETL DAG tutorial',
        schedule_interval='0 10 * 10 *',catchup=False,tags=['nba_stats']) as dag:

        dag.doc_md = __doc__

        def extract(**kwargs):
                ti = kwargs['ti']

                bs = Box_scores()
                year = int(strftime("%Y"))

                start_reg = 9
                end_reg = 4
                start_post = end_reg
                end_post = 7
                reg_season = True

                month = int(strftime('%m'))
                year_range = str(year-1) + "-{:0>2d}".format(get_last_year(year))

                ti = kwargs['ti']
                if month <= end_reg or month >= start_reg:

                        url = bs.build_url(year_range, not reg_season)

                elif month >= start_post and month <= end_post:

                        url = bs.build_url(year_range, not reg_season)
                else:
                        return # exit dag
                
                return bs.iter_all(url,)

    


In [ ]:
def get_last_year(year):
        if(year > 2000):
            return year % 2000
        return year % 100


In [ ]:
import pendulum

In [ ]:
d = pendulum.datetime(2022,5,27)

In [ ]:
link = 'https://www.nba.com/stats/players/boxscores/?Season=2021-22&SeasonType=Regular%20Season&DateFrom=05%2F01%2F2021&DateTo=05%2F27%2F2022'

In [ ]:
import datetime
from datetime import timedelta

d = datetime.datetime

e = d.strftime(datetime.date.today(),'%m-%d-%Y').replace('-','%2F')

In [ ]:
s = link.find(e)
link[s:len(e)+s]

In [ ]:
e

In [ ]:
def extract(**kwargs):
# ti = kwargs['ti']
        d = datetime.datetime
        from web_scraper import Box_scores
        bs = Box_scores()
        year = int(d.strftime(datetime.date.today(),"%Y"))
        
        s = datetime.date.today() - timedelta(days= 13)

        start_reg = 9
        end_reg = 4
        start_post = end_reg
        end_post = 7
        reg_season = True

        month = int(strftime('%m'))
        year_range = str(year-1) + "-{:0>2d}".format(get_last_year(year))

        # ti = kwargs['ti']
        if month <= end_reg or month >= start_reg:

                url = bs.build_url(year_range, not reg_season)

        elif month >= start_post and month <= end_post:

                url = bs.build_url(year_range,s, not reg_season)
        else:
                return # exit dag

        return bs.iter_all(url)


In [ ]:
pages = []
for i in extract():
    pages.append(i)

In [ ]:
def get_player_and_team_ids(self,html):
            soup = BeautifulSoup(html, 'html.parser')
            table = soup.find("table")
            
            pids = []
            tids = []
            gids = []
            
            player_reg = '/player/\d+'
            team_reg = '/team/\d+'
            game_reg = '/game/\d+'

            for l in table.find_all('a'):
                player_match = search(player_reg,l.get('href'))

                team_match = search(team_reg,l.get('href'))
                
                game_match = search(game_reg,l.get('href'))

                if(player_match):
                    player_id = int(search('\d+',player_match.group()).group())
                    pids.append(player_id)
                elif(team_match):
                    team_id = int(search('\d+',team_match.group()).group())
                    tids.append(team_id)
                elif(game_match):
                    game_id = search('\d+',game_match.group()).group()
                    gids.append(game_id)
                    
            return pids, tids, gids

In [ ]:
import pandas as pd
from bs4 import BeautifulSoup

In [ ]:
df = pd.DataFrame()
for i in pages:
    df = pd.concat((df,pd.read_html(i)[0]))

In [ ]:
df.drop(columns=['Season','FP','FG%','FT%','3P%','REB'],inplace = True)

In [ ]:
df.columns

In [ ]:
di = {'3pm':'pm3', '3pa':'pa3','match\xa0up':'match_up','min':'mins',
'w/l': 'result','+/-':'plus_minus','game\xa0date':'game_date'}
# dicto = dict(df.columns,)
transform = [i.lower() for i in df.columns]
df.rename(str.lower,axis='columns',inplace=True)

df.rename(columns=di,inplace =True)

In [ ]:
df.dtypes

In [ ]:
with open('shots.html','r') as f:
    h = f.read()

In [ ]:
soup = BeautifulSoup(h)

In [ ]:
soup.find('th')

In [ ]:
def build_url(self,year,s,reg_season = False):
            form = '%m-%d-%Y'
            d = datetime.datetime
            e = datetime.date.today() 

            s = d.strftime(s,form)
            e = d.strftime(e,form)
            start = '&DateFrom='+ s.replace('-','%2F')
            end = '&DateTo='+ e.replace('-','%2F')

In [ ]:
from bs4 import BeautifulSoup
import re
from selenium.webdriver.support.select import Select
from selenium.webdriver.common.by import By
from selenium import webdriver
from selenium.webdriver.support.select import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup

from re import search
import pandas as pd


In [ ]:
class Box_scores:
        def __init__(self):
            self.REG = "Regular%20Season"
            self.POST = "Playoffs"
            self.root = "https://www.nba.com/stats/players/boxscores/?" # + [teamname -prefix_1] +
            self.mid = "Season=" # +[year] YYYY-YY +
            self.tail = "&SeasonType=" # + [Regular%20Season|Playoffs]
            self.xpath = "/html/body/main/div/div/div[2]/div/div/nba-stat-table/div[2]/div[1]/table"
            self.select_xpath = "/html/body/main/div/div/div[2]/div/div/nba-stat-table/div[1]/div/div/select"
            
        def build_url(self,year,reg_season = False):
            if(reg_season):
                return self.root + self.mid + year + self.tail + self.REG
            
            return self.root + self.mid + year + self.tail + self.POST
        
        
        def iter_all(self, url):
            wait = 20
            with webdriver.Chrome() as driver:
                driver.get(url)
                element = WebDriverWait(driver,wait).until(EC.presence_of_element_located((By.XPATH,self.xpath)))
                s = driver.find_element(By.XPATH,self.select_xpath)
                t = s.text
                t = t.split("\n")

                
                for i in t[1:]:
                    s = driver.find_element(By.XPATH,self.select_xpath)
                    s = Select(s)
                    s.select_by_visible_text(i)
                    yield driver.page_source
                

            return
            
        
        def get_player_and_team_ids(self,html):
            soup = BeautifulSoup(html, 'html.parser')
            table = soup.find("table")
            
            pids = []
            tids = []
            gids = []
            
            player_reg = '/player/\d+'
            team_reg = '/team/\d+'
            game_reg = '/game/\d+'

            for l in table.find_all('a'):
                player_match = search(player_reg,l.get('href'))

                team_match = search(team_reg,l.get('href'))
                
                game_match = search(game_reg,l.get('href'))

                if(player_match):
                    player_id = int(search('\d+',player_match.group()).group())
                    pids.append(player_id)
                elif(team_match):
                    team_id = int(search('\d+',team_match.group()).group())
                    tids.append(team_id)
                elif(game_match):
                    game_id = search('\d+',game_match.group()).group()
                    gids.append(game_id)
                    
            return pids, tids, gids


In [ ]:
def extract(year, regular):
    # ti = kwargs['ti']
    d = datetime.datetime
    bs = Box_scores()

    year_range = str(year-1) + "-{:0>2d}".format(get_last_year(year))

    url = bs.build_url(year_range, regular)

    return bs.iter_all(url)


In [116]:
for year in range(2014,2011,-1):
    # playoffs
    # j = 0
    # for i in extract(year, False):
    #     with open('../../../data/'+str(year)+'_playoffs_'+str(j)+'.html','w') as f:
    #         f.write(i)
    #     j+=1
    
    j=0
    for i in extract(year, True):
        with open('../../../data/'+str(year)+'_reg_season_'+str(j)+'.html','w') as f:
            f.write(i)
        j+=1


TimeoutException: Message: 
Stacktrace:
#0 0x00557d7f2988 <unknown>
#1 0x00557d5b541c <unknown>
#2 0x00557d5e0c18 <unknown>
#3 0x00557d6063f4 <unknown>
#4 0x00557d5d856c <unknown>
#5 0x00557d5d9128 <unknown>
#6 0x00557d81e5a4 <unknown>
#7 0x00557d83547c <unknown>
#8 0x00557d820238 <unknown>
#9 0x00557d835d58 <unknown>
#10 0x00557d8147c0 <unknown>
#11 0x00557d84e320 <unknown>
#12 0x00557d84e474 <unknown>
#13 0x00557d86597c <unknown>
#14 0x007f990ad628 <unknown>
#15 0x007f98bda01c <unknown>


In [125]:
from re import search

In [132]:
def get_player_and_team_ids(html):
            soup = BeautifulSoup(html, 'html.parser')
            table = soup.find("table")
            
            pids = []
            tids = []
            gids = []
            
            player_reg = '/player/\d+'
            team_reg = '/team/\d+'
            game_reg = '/game/\d+'

            for l in table.find_all('a'):
                player_match = search(player_reg,l.get('href'))

                team_match = search(team_reg,l.get('href'))
                
                game_match = search(game_reg,l.get('href'))

                if(player_match):
                    player_id = int(search('\d+',player_match.group()).group())
                    pids.append(player_id)
                elif(team_match):
                    team_id = int(search('\d+',team_match.group()).group())
                    tids.append(team_id)
                elif(game_match):
                    game_id = search('\d+',game_match.group()).group()
                    gids.append(game_id)
                    
            return pids, tids, gids


In [141]:
from os import listdir

In [144]:
fp = '../../../data/'
df = pd.DataFrame()
for i in listdir(fp):
    if '2022_playoffs' in i:
        with open(fp+i,'r') as f:
            html = f.read()
        pids, tids, gids = get_player_and_team_ids(html)

        right = pd.read_html(html)[0]
        right['player_id'] = pids
        right['tean_id'] = pids
        right['game_id'] = pids
        df= pd.concat((df,right))

In [149]:
def players(df):
    d = df[['player_id','Player']]
    d.rename(columns={'Player':'name'},inplace=True)

    return d

In [151]:
players = players(df)

/tmp/ipykernel_327504/3369944772.py:3 SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

In [156]:
players.drop_duplicates('player_id').dtypes

player_id     int64
name         object
dtype: object

array([ 202691,  203490, 1626172,  202722,  203506,  202710,  200782,
       1629639,  203473, 1630202, 1629684, 1627759,  203935,  201143,
       1630209, 1630174, 1629752,  203917, 1628369, 1628389, 1628436,
       1628464, 1628997, 1629057, 1629130, 1629216, 1629312, 1629622,
        201939,  203504, 1630228, 1629673,  203952,  203110, 1629029,
       1628467, 1628373,  203939, 1627737, 1627814, 1627827, 1628973,
       1630182, 1630541, 1628425, 1629308, 1628401,  203648, 1626192,
        203507,  202083,  201586,  203937, 1626156, 1629020,  202328,
       1630217,  201976, 1629630, 1630162, 1630214, 1626145, 1628415,
       1630183, 1629723, 1626157, 1627736, 1628991, 1629162, 1629634,
        203497,  203493, 1626220, 1628378, 1629672,  201144,  202355,
        203903,  203915, 1627823, 1627863,  202711, 1626164,  203468,
        202685,  101108,  202687,  201580,  201988, 1630549, 1629651,
       1628971,  203083, 1630556, 1630560,  201609,  202681,  203552,
        203109, 1626